In [52]:
#%%
import sys
#sys.path.insert(0, "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages")
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import os
import numpy as np
import tqdm
from sklearn.model_selection import train_test_split
import random
import tensorflow as tf
from sklearn import preprocessing 
from sklearn.metrics import confusion_matrix, f1_score
import pickle
import itertools

In [53]:
##### Create Fake Test Set ########
exmp_test_summary = pd.read_csv("/home/ubuntu/birdsong-recognition/example_test_audio_summary.csv")

In [54]:
exmp_test_summary['audio_id'] = np.where(exmp_test_summary['filename'] == "BLKFR-10-CPL", exmp_test_summary['filename'] + "_20190611_093000", exmp_test_summary['filename'] + "_20190606_093000")

In [55]:
exmp_test_summary['site'] = np.where(exmp_test_summary['filename'] == "BLKFR-10-CPL", "site_1", "site_3")

In [56]:
exmp_test_summary = exmp_test_summary.rename(columns={"filename_seconds": "row_id"})

In [57]:
test_df = exmp_test_summary[['site', 'row_id', 'seconds', 'audio_id']]

In [62]:
test_df.shape

(153, 4)

In [59]:
base_path = "/home/ubuntu/birdsong-recognition/"
test_path = "/home/ubuntu/birdsong-recognition/example_test_audio/"

In [60]:
print(tf.__version__)

1.15.0


In [61]:
model = tf.keras.models.load_model('full_model_resnet_30epoch.h5')

In [62]:
le = preprocessing.LabelEncoder()
le.classes_ = np.load('classes.npy')
test_df = pd.read_csv(base_path+'test.csv')

In [6]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 128, 431, 1) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 437, 1)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 216, 64)  3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 216, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [63]:
unique_audio_id = test_df.audio_id.unique()

In [120]:
test_df

,site,row_id,seconds,audio_id
0,site_1,BLKFR-10-CPL_20190611_093000_5,5,BLKFR-10-CPL_20190611_093000
1,site_1,BLKFR-10-CPL_20190611_093000_10,10,BLKFR-10-CPL_20190611_093000
2,site_1,BLKFR-10-CPL_20190611_093000_15,15,BLKFR-10-CPL_20190611_093000
3,site_1,BLKFR-10-CPL_20190611_093000_20,20,BLKFR-10-CPL_20190611_093000
4,site_1,BLKFR-10-CPL_20190611_093000_25,25,BLKFR-10-CPL_20190611_093000
...,...,...,...,...
148,site_3,ORANGE-7-CAP_20190606_093000_280,280,ORANGE-7-CAP_20190606_093000
149,site_3,ORANGE-7-CAP_20190606_093000_285,285,ORANGE-7-CAP_20190606_093000
150,site_3,ORANGE-7-CAP_20190606_093000_290,290,ORANGE-7-CAP_20190606_093000
151,site_3,ORANGE-7-CAP_20190606_093000_295,295,ORANGE-7-CAP_20190606_093000


In [125]:
for audio_id in unique_audio_id:
    df = test_df.query(
            f"audio_id == '{audio_id}'").reset_index(drop=True)
    print(df.site.unique())

    #for idx in df.index:
    #    sample = df.loc[idx, :]
    #    site = sample.site
    #    row_id = sample.row_id

    #    print(sample)

['site_1']
['site_3']


In [144]:
SR = 44100
unique_audio_id = test_df.audio_id.unique()

preds = {}
out = []
for audio_id in unique_audio_id:
    clip, _ = librosa.load(test_path + "/" + (audio_id + ".mp3"),
                                   sr=SR,
                                   mono=True,
                                   res_type="kaiser_fast")

    df = test_df.query(
            f"audio_id == '{audio_id}'").reset_index(drop=True)
    site = df.site.unique()[0]

    if site == "site_3":
            print(f'{site} Site and Audio ID {audio_id}')
            y = clip
            len_y = len(y)
            start = 0
            end = SR * 5
            spectograms = []
            preds_site3 = []
            while len_y > start:
                y_batch = y[start:end]
                if len(y_batch) != (SR * 5):
                    break
                start = end
                end = end + SR * 5

                melspec = librosa.feature.melspectrogram(y_batch,
                                                             sr=SR,
                                                             n_mels = 128)
                melspec = librosa.amplitude_to_db(melspec)
                #spectograms.append(melspec)
                spectograms_norm = librosa.util.normalize(melspec)
                spectograms_norm = spectograms_norm[np.newaxis,:,:,np.newaxis]
                pred = model.predict(spectograms_norm)
                pred = np.argwhere(pred>0.6)
                
                if pred.size == 0:
                    #preds_site3.append('nocall')
                    continue
                else:
                    pred = le.inverse_transform(np.ravel(pred))
                    print(pred)
                    preds_site3.append(pred)
            
            flatten = itertools.chain.from_iterable
            preds_site3 = list(flatten(preds_site3))
            preds_unique = set(preds_site3)
            preds_unique = ' '.join(preds_unique)
            preds[audio_id] = preds_unique
            #spectograms = np.stack(spectograms)
    else:
        print(f'{site} Site and Audio ID {audio_id}')
        for idx in tqdm.tqdm_notebook(df[df['audio_id'] == audio_id].index):
            sample = df.loc[idx, :]
            site = sample.site
            row_id = sample.row_id
            
            print(row_id)
            
            end_seconds = int(sample.seconds)
            start_seconds = int(end_seconds - 5)

            start_index = SR * start_seconds
            end_index = SR * end_seconds

            y = clip[start_index:end_index]

            melspec = librosa.feature.melspectrogram(y, sr=SR, n_mels = 128)
            spectograms = librosa.amplitude_to_db(melspec)
            spectograms_norm = librosa.util.normalize(spectograms)
            spectograms_norm = spectograms_norm[np.newaxis,:,:,np.newaxis]
            pred = model.predict(spectograms_norm)
            pred = np.argwhere(pred>0.6)
            
            if pred.size == 0:
                preds[row_id] = 'nocall'
            else:
                pred = le.inverse_transform(np.ravel(pred))
                pred = ' '.join(pred)
                preds[row_id] = pred

    #out.append(spectograms)

/home/ubuntu/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


site_1 Site and Audio ID BLKFR-10-CPL_20190611_093000


/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:57: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


BLKFR-10-CPL_20190611_093000_5
BLKFR-10-CPL_20190611_093000_10
BLKFR-10-CPL_20190611_093000_15
BLKFR-10-CPL_20190611_093000_20
BLKFR-10-CPL_20190611_093000_25
BLKFR-10-CPL_20190611_093000_30
BLKFR-10-CPL_20190611_093000_35
BLKFR-10-CPL_20190611_093000_40
BLKFR-10-CPL_20190611_093000_45
BLKFR-10-CPL_20190611_093000_50
BLKFR-10-CPL_20190611_093000_55
BLKFR-10-CPL_20190611_093000_60
BLKFR-10-CPL_20190611_093000_65
BLKFR-10-CPL_20190611_093000_70
BLKFR-10-CPL_20190611_093000_75
BLKFR-10-CPL_20190611_093000_80
BLKFR-10-CPL_20190611_093000_85
BLKFR-10-CPL_20190611_093000_90
BLKFR-10-CPL_20190611_093000_95
BLKFR-10-CPL_20190611_093000_100
BLKFR-10-CPL_20190611_093000_105
BLKFR-10-CPL_20190611_093000_110
BLKFR-10-CPL_20190611_093000_115
BLKFR-10-CPL_20190611_093000_120
BLKFR-10-CPL_20190611_093000_125
BLKFR-10-CPL_20190611_093000_130
BLKFR-10-CPL_20190611_093000_135
BLKFR-10-CPL_20190611_093000_140
BLKFR-10-CPL_20190611_093000_145
BLKFR-10-CPL_20190611_093000_150
BLKFR-10-CPL_20190611_093000_1

/home/ubuntu/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


site_3 Site and Audio ID ORANGE-7-CAP_20190606_093000
['aldfly' 'yebsap']
['aldfly' 'buggna']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'yebsap']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'hoomer']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'hoomer']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'amered']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldfly' 'buwwar']
['aldf

In [148]:
sub = pd.DataFrame(preds.items(), columns=['row_id', 'birds'])

In [149]:
sub.to_csv("submission.csv", index=False)

,row_id,birds
0,BLKFR-10-CPL_20190611_093000_5,aldfly horlar
1,BLKFR-10-CPL_20190611_093000_10,aldfly horlar
2,BLKFR-10-CPL_20190611_093000_15,aldfly horlar
3,BLKFR-10-CPL_20190611_093000_20,nocall
4,BLKFR-10-CPL_20190611_093000_25,nocall
...,...,...
89,BLKFR-10-CPL_20190611_093000_450,nocall
90,BLKFR-10-CPL_20190611_093000_455,nocall
91,BLKFR-10-CPL_20190611_093000_460,nocall
92,BLKFR-10-CPL_20190611_093000_465,nocall


In [116]:
############################ Test on batch ##################################

out = np.vstack(out)

In [68]:
out_norm = [librosa.util.normalize(i) for i in out]

In [117]:
out.shape

(155, 128, 431)

In [69]:
out_norm = np.array(out_norm)

In [70]:
out_norm = out_norm[:,:,:,np.newaxis]

In [71]:
out_norm.shape

(155, 128, 431, 1)

In [72]:
pred = model.predict(out_norm)

In [101]:
x = np.argwhere(pred[125]>0.6)

In [102]:
x = le.inverse_transform(np.ravel(x))

In [103]:
' '.join(x)

'hoomer'

In [86]:
if x.size == 0:
    print('yes')

In [73]:
np.argwhere(pred>0.6).shape

(62, 2)

In [101]:
np.argmax(pred , axis = -1).shape

(155,)

In [93]:
pred[7].round(3)

array([0.   , 0.   , 0.   , 0.152, 0.   , 0.011, 0.   , 0.001, 0.   ,
       0.   , 0.   , 0.001, 0.003, 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.006, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.077, 0.   , 0.   , 0.001, 0.   , 0.   , 0.033, 0.   ,
       0.   , 0.   , 0.017, 0.   , 0.145, 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.015, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001,
       0.001, 0.003, 0.006, 0.004, 0.   , 0.   , 0.   , 0.   , 0.009,
       0.001, 0.   , 0.   , 0.006, 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.003, 0.   , 0.   , 0.   , 0.007, 0.017, 0.   , 0.   , 0.   ,
       0.002, 0.   , 0.005, 0.001, 0.   , 0.   , 0.003, 0.001, 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.016, 0.   , 0.001, 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.03 , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.002, 0.008, 0.   , 0.   , 0.001, 0.002, 0.   , 0.015,
       0.   , 0.   ,